In [6]:
import pandas as pd
import numpy as np
import random
from itertools import product
import random
import itertools
with open(f"NumPart/Part50Instance1") as file:
    data = list(map(int, file.readlines()))
data = data[1:]

In [7]:
def grasp_function_izaro_v2(numbers):
    s_data = sorted(numbers, reverse=True)
    half_indexes = random.sample(range(int(len(numbers)/2)), int(len(numbers)/2))
    pair_indexes = [(index*2, index*2 +1) for index in half_indexes]
    a_mul = 1
    b_mul = 1
    chosen = np.zeros(len(numbers))
    for i in pair_indexes:
        index = 0 if a_mul < b_mul else 1
        num_ind_1 = numbers.index(s_data[i[index]])
        num_ind_2 = numbers.index(s_data[i[1 - index]]) 
        chosen[num_ind_1] = 1
        a_mul *= numbers[num_ind_1]
        b_mul *= numbers[num_ind_2]
        
    return  list(map(int, list(chosen)))


def grasp_function_imanol(numbers,k=3):
    s_data = sorted(numbers, reverse=True)
    a_mul = 1
    b_mul = 1
    chosen = np.zeros(len(numbers))
    for i in range(int(len(numbers)/2)):
        if(k > len(s_data)):
            k = len(s_data)

        sel_ind = random.sample(range(k), 2)
        sel_ind = sorted(sel_ind, reverse=True)
        
        num_ind_1 = numbers.index(s_data[sel_ind[0]])
        num_ind_2 = numbers.index(s_data[sel_ind[1]])
        
        del s_data[sel_ind[0]]
        del s_data[sel_ind[1]]
        
        group = 0 if a_mul < b_mul else 1
        max_ind = num_ind_2
        min_ind = num_ind_1
        if(numbers[num_ind_1] > numbers[num_ind_2]):
            max_ind = num_ind_1
            min_ind = num_ind_2
            
        chosen[max_ind] = group
        chosen[min_ind] = 1-group
        a_mul *= (numbers[max_ind]**(1-group) + numbers[min_ind]**(group))
        b_mul *= (numbers[max_ind]**(group) + numbers[min_ind]**(1-group))    
    
    return  list(map(int, list(chosen)))


In [8]:
def objective_function_optim(a, b, num_a, num_b):
    new_a = int(a/num_a) * num_b
    new_b = int(b/num_b) * num_a
    return abs(new_a - new_b), new_a, new_b

def objective_function(data, chosen):
    a1 = 1
    b1 = 1
    for a in range(len(data)):
        a1 *= data[a]**chosen[a]
        b1 *= data[a]**(1-chosen[a])
    return abs(a1 - b1), a1, b1


In [25]:
def get_initial_solution(numbers):
    initial = random.sample(range(len(numbers)), len(numbers))
    init_chosen = np.zeros(len(initial))
    init_chosen[initial[:int(len(initial)/2)]] = 1
    return list(map(int, list(init_chosen)))

def local_search(numbers, chosen, best, a, b):
    while True:
        local_best_chosen, local_best, a, b  = get_best_neighbor(numbers, chosen, best, a, b)
        if local_best_chosen == chosen:
            break
        chosen = local_best_chosen[:]
        best = local_best
    return chosen, best


def local_search_multi(numbers, multistart=5, init_fn = grasp_function_izaro_v2,**kwargs ):
    chosen = init_fn(numbers, **kwargs)
    best, a, b =  objective_function(numbers, chosen)
    chosen_list = []
    best_list = []
    for _ in range(multistart):
        chosen, best = local_search(numbers, chosen, best, a, b)
        chosen_list.append(chosen)
        best_list.append(best)
        
        # We get a initial solution for the multistart
        chosen = init_fn(numbers, **kwargs)
        best, a, b =  objective_function(numbers, chosen)

    index_min = np.argmin(best_list)

    return  best_list[index_min]


def get_best_neighbor(numbers, chosen, best, a, b):
    best_a = a
    best_b = b
    best_ = best
    best_chosen = chosen[:]
    one_indexes = [ i for i, x in enumerate(chosen) if x == 1 ] 
    zero_indexes = [ i for i, x in enumerate(chosen) if x == 0 ] 
    
    for zero_index in zero_indexes:
        for one_index in one_indexes:
            chosen_ = chosen[:]
            chosen_[zero_index] = 1
            chosen_[one_index] = 0
            value, var_a, var_b = objective_function_optim(a, b, numbers[one_index], numbers[zero_index])
            if value < best_:
                best_ = value
                best_chosen = chosen_[:]
                best_a = var_a
                best_b = var_b

    return best_chosen, best_, best_a, best_b
        

In [26]:

local_search_multi(data, multistart=10,)


12743488653

In [27]:
def delete_redundant(chosen):
  def_chosen = []
  for a in chosen:
    a_alt = [int(not x) for x in a]
    if a not in def_chosen and a_alt not in def_chosen:
      def_chosen.append(a)
  return def_chosen

def get_all_combinations(n, redundant=False):
  x = [list(i) for i in product(range(2), repeat=n) if sum(i) == int(n/2)]
  if not redundant: 
    x = delete_redundant(x)
  return x



In [41]:
from collections import defaultdict
def calculate_best_deterministic(numbers):
    n = len(numbers)
    best_score = float("inf")
    for comb in product(range(2), repeat=n):
        if sum(comb)*2 != n:
            continue
        score, _, _ = objective_function(numbers, comb)
        if score < best_score:
            best_score = score
            best_comb = comb
    return best_score


def get_best_of_execution(results):
    value_list = list(zip(*results))
    index_min = np.argmin(value_list[1])
    return value_list[0][index_min], value_list[1][index_min]   

instances = range(1, 11)
numbers =  [10]
best_deterministic = {}
best_izaro = defaultdict(int)
best_imanol = defaultdict(int)
best_random = defaultdict(int)
for _ in range(20):
    for num in numbers:
        for instance in instances:
            with open(f"NumPart/Part{num}Instance{instance}") as file:
                data = list(map(int, file.readlines()))
                data = data[1:]
                det = calculate_best_deterministic(data)
                izaro = local_search_multi(data, multistart=100)
                imanol = local_search_multi(data, multistart=100, init_fn=grasp_function_imanol)
                rand =  local_search_multi(data, multistart=100, init_fn=get_initial_solution)
                best_deterministic[f"{num}_{instance}"] = det
                best_izaro[f"{num}_{instance}"] +=  izaro == det
                best_imanol[f"{num}_{instance}"] += imanol == det
                best_random[f"{num}_{instance}"] += rand == det


In [42]:

pd.DataFrame([best_deterministic, best_izaro, best_imanol, best_random], index=["Deterministic", "Pairs", "K-max", "Random"]).T

,Deterministic,Pairs,K-max,Random
10_1,269114681010,20,20,20
10_2,118479464640,20,20,20
10_3,202764848496,20,20,20
10_4,1234214659656,20,20,20
10_5,16768974600,20,20,20
10_6,107714458776,20,20,20
10_7,2041905009816,20,20,20
10_8,10048311000,20,20,20
10_9,60287671392,0,20,20
10_10,12743488653,20,20,20


In [75]:
instances = range(1, 11)
numbers =  [100]
best_izaro = defaultdict(int)
best_imanol = defaultdict(int)
best_random = defaultdict(int)
for _ in range(10):
    for num in numbers:
        print(num)
        for instance in instances:
            with open(f"NumPart/Part{num}Instance{instance}") as file:
                data = list(map(int, file.readlines()))
                data = data[1:]
                izaro = local_search_multi(data, multistart=20)
                imanol = local_search_multi(data, multistart=20, init_fn=grasp_function_imanol)
                rand =  local_search_multi(data, multistart=20, init_fn=get_initial_solution)
                best = min([izaro, imanol, rand])
                best_izaro[f"{num}_{instance}"] +=  (izaro == best)
                best_imanol[f"{num}_{instance}"] += (imanol == best)
                best_random[f"{num}_{instance}"] += (rand == best)


100
100
100
100
100
100
100
100
100
100


In [76]:
pd.DataFrame([ best_izaro, best_imanol, best_random], index=[ "Pairs", "K-max", "Random"]).T

,Pairs,K-max,Random
100_1,4,4,2
100_2,4,3,3
100_3,6,2,2
100_4,5,3,2
100_5,5,2,3
100_6,4,2,4
100_7,6,3,1
100_8,4,2,4
100_9,5,2,3
100_10,3,5,2


In [69]:
pd.DataFrame([ best_izaro, best_imanol, best_random], index=[ "Pairs", "K-max", "Random"]).T

,Pairs,K-max,Random
50_1,2,3,5
50_2,3,1,6
50_3,4,2,4
50_4,3,4,3
50_5,1,4,5
50_6,5,3,2
50_7,2,3,5
50_8,5,3,2
50_9,4,3,3
50_10,5,4,1
